# TextSQL RAG Pipeline 學習筆記

本筆記本將詳細介紹如何構建一個完整的 textSQL RAG (Retrieval-Augmented Generation) 流水線。

## 目錄
1. [RAG 概念介紹](#1-rag-概念介紹)
2. [TextSQL 基礎](#2-textsql-基礎)
3. [環境設置](#3-環境設置)
4. [數據預處理](#4-數據預處理)
5. [向量化與索引](#5-向量化與索引)
6. [檢索系統](#6-檢索系統)
7. [SQL 生成](#7-sql-生成)
8. [完整流水線](#8-完整流水線)
9. [評估與優化](#9-評估與優化)
10. [實際應用案例](#10-實際應用案例)

## 1. RAG 概念介紹

### 什麼是 RAG？
RAG（Retrieval-Augmented Generation）是一種結合了檢索和生成的架構：
1. **檢索階段**：從知識庫中找到相關信息
2. **生成階段**：基於檢索到的信息生成回答

### TextSQL RAG 的特點
- 專注於自然語言到 SQL 查詢的轉換
- 結合數據庫模式信息
- 支持複雜的數據庫查詢生成

In [1]:
# 基本概念示例
import pandas as pd
import numpy as np
from typing import List, Dict, Any

# RAG 流程示意
class SimpleRAGConcept:
    def __init__(self):
        self.knowledge_base = []
        self.query_history = []
    
    def retrieve(self, query: str) -> List[str]:
        """檢索相關信息"""
        # 這裡是簡化的檢索邏輯
        relevant_docs = [doc for doc in self.knowledge_base 
                        if any(word.lower() in doc.lower() for word in query.split())]
        return relevant_docs[:3]  # 返回前3個相關文檔
    
    def generate(self, query: str, context: List[str]) -> str:
        """基於上下文生成回答"""
        # 這裡是簡化的生成邏輯
        return f"基於上下文 {context} 對查詢 '{query}' 的回答"
    
    def rag_pipeline(self, query: str) -> str:
        """完整的RAG流水線"""
        # 1. 檢索
        relevant_context = self.retrieve(query)
        # 2. 生成
        response = self.generate(query, relevant_context)
        # 3. 記錄查詢歷史
        self.query_history.append({'query': query, 'response': response})
        return response

# 示例使用
rag_demo = SimpleRAGConcept()
rag_demo.knowledge_base = [
    "用戶表包含用戶ID、姓名、郵箱等字段",
    "訂單表記錄了所有的購買信息",
    "產品表存儲產品的詳細信息"
]

result = rag_demo.rag_pipeline("如何查詢用戶的訂單信息？")
print("RAG 示例結果:")
print(result)

RAG 示例結果:
基於上下文 [] 對查詢 '如何查詢用戶的訂單信息？' 的回答


## 2. TextSQL 基礎

### 核心概念
TextSQL 是將自然語言查詢轉換為 SQL 語句的過程。關鍵組件包括：
- **Schema Understanding**: 理解數據庫結構
- **Intent Recognition**: 識別用戶意圖
- **SQL Generation**: 生成對應的SQL語句

In [2]:
# TextSQL 基礎組件
class DatabaseSchema:
    """數據庫模式類"""
    def __init__(self):
        self.tables = {}
        self.relationships = []
    
    def add_table(self, table_name: str, columns: Dict[str, str]):
        """添加表結構"""
        self.tables[table_name] = columns
    
    def add_relationship(self, table1: str, column1: str, table2: str, column2: str):
        """添加表關係"""
        self.relationships.append({
            'from_table': table1,
            'from_column': column1,
            'to_table': table2,
            'to_column': column2
        })
    
    def get_schema_info(self) -> str:
        """獲取模式信息"""
        schema_info = "數據庫模式信息:\n"
        for table, columns in self.tables.items():
            schema_info += f"表 {table}: {columns}\n"
        return schema_info

# 創建示例數據庫模式
schema = DatabaseSchema()
schema.add_table('users', {
    'user_id': 'INT PRIMARY KEY',
    'name': 'VARCHAR(100)',
    'email': 'VARCHAR(100)',
    'created_at': 'DATETIME'
})

schema.add_table('orders', {
    'order_id': 'INT PRIMARY KEY',
    'user_id': 'INT',
    'product_id': 'INT',
    'quantity': 'INT',
    'order_date': 'DATETIME'
})

schema.add_table('products', {
    'product_id': 'INT PRIMARY KEY',
    'name': 'VARCHAR(100)',
    'price': 'DECIMAL(10,2)',
    'category': 'VARCHAR(50)'
})

schema.add_relationship('orders', 'user_id', 'users', 'user_id')
schema.add_relationship('orders', 'product_id', 'products', 'product_id')

print(schema.get_schema_info())

數據庫模式信息:
表 users: {'user_id': 'INT PRIMARY KEY', 'name': 'VARCHAR(100)', 'email': 'VARCHAR(100)', 'created_at': 'DATETIME'}
表 orders: {'order_id': 'INT PRIMARY KEY', 'user_id': 'INT', 'product_id': 'INT', 'quantity': 'INT', 'order_date': 'DATETIME'}
表 products: {'product_id': 'INT PRIMARY KEY', 'name': 'VARCHAR(100)', 'price': 'DECIMAL(10,2)', 'category': 'VARCHAR(50)'}



## 3. 環境設置

### 安裝必要的庫

In [ ]:
# 在 Kaggle 環境中安裝必要的包
import sys
import subprocess

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# 首先修復protobuf版本衝突問題
print("🔧 修復protobuf版本衝突...")
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "protobuf"])
    print("✓ protobuf 版本已修復")
except Exception as e:
    print(f"⚠️ protobuf 修復失敗，將使用備用方案: {e}")

# 核心包列表 (sqlite3 是Python標準庫，不需要安裝)
packages = [
    'transformers',
    'torch',
    'sentence-transformers',
    'faiss-cpu',
    'sqlparse',
    'datasets'  # 用於加載 Hugging Face 數據集
]

# 可選包列表（如果安裝失敗不影響核心功能）
optional_packages = [
    'chromadb',
    'langchain',
    'openai'
]

print("\n正在安裝核心包...")
for package in packages:
    try:
        install_package(package)
        print(f"✓ {package} 安裝成功")
    except Exception as e:
        print(f"✗ {package} 安裝失敗: {e}")

print("\n正在安裝可選包...")
for package in optional_packages:
    try:
        install_package(package)
        print(f"✓ {package} 安裝成功")
    except Exception as e:
        print(f"⚠️ {package} 安裝失敗（不影響核心功能）: {e}")

print("\n環境設置完成！")
print("📝 注意：sqlite3 是 Python 內建庫，無需安裝")

In [ ]:
# 導入必要的庫（帶錯誤處理）
import os
import json
import sqlite3
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Tuple
import warnings
warnings.filterwarnings('ignore')

# 核心導入
print("導入核心庫...")
try:
    import sqlparse
    print("✓ sqlparse 導入成功")
except ImportError as e:
    print(f"⚠️ sqlparse 導入失敗: {e}")
    # 創建簡單的備用解析器
    class MockSQLParse:
        @staticmethod
        def parse(sql):
            return [type('Token', (), {'tokens': [sql] if sql.strip() else []})]
    sqlparse = MockSQLParse()

# 向量化和檢索（帶備用方案）
print("導入向量化庫...")
try:
    from sentence_transformers import SentenceTransformer
    print("✓ sentence_transformers 導入成功")
    SENTENCE_TRANSFORMERS_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ sentence_transformers 導入失敗: {e}")
    SENTENCE_TRANSFORMERS_AVAILABLE = False
    # 將在TextEmbedder中使用備用方案

try:
    import faiss
    print("✓ faiss 導入成功")
    FAISS_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ faiss 導入失敗: {e}")
    FAISS_AVAILABLE = False
    # 將使用簡單的向量搜索備用方案

# 自然語言處理（可選）
print("導入NLP庫...")
try:
    from transformers import AutoTokenizer, AutoModel
    import torch
    print("✓ transformers 和 torch 導入成功")
    TRANSFORMERS_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ transformers/torch 導入失敗: {e}")
    TRANSFORMERS_AVAILABLE = False

print("\n✅ 庫導入完成！")
print(f"📊 功能狀態:")
print(f"  • 句子嵌入: {'可用' if SENTENCE_TRANSFORMERS_AVAILABLE else '使用備用方案'}")
print(f"  • 向量索引: {'可用' if FAISS_AVAILABLE else '使用備用方案'}")
print(f"  • 高級NLP: {'可用' if TRANSFORMERS_AVAILABLE else '基礎功能'}")
print(f"  • SQL解析: 可用")
print(f"  • 數據庫: 可用 (SQLite3)")

## 4. 數據預處理

### 創建示例數據庫和數據

In [ ]:
# 創建示例 SQLite 數據庫
def create_sample_database():
    """創建示例數據庫"""
    conn = sqlite3.connect('sample_ecommerce.db')
    cursor = conn.cursor()
    
    # 創建用戶表
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        user_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        created_at DATETIME DEFAULT CURRENT_TIMESTAMP
    )
    ''')
    
    # 創建產品表
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS products (
        product_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        price DECIMAL(10,2) NOT NULL,
        category TEXT NOT NULL,
        description TEXT
    )
    ''')
    
    # 創建訂單表
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS orders (
        order_id INTEGER PRIMARY KEY,
        user_id INTEGER,
        product_id INTEGER,
        quantity INTEGER NOT NULL,
        order_date DATETIME DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (user_id) REFERENCES users (user_id),
        FOREIGN KEY (product_id) REFERENCES products (product_id)
    )
    ''')
    
    # 插入示例數據
    users_data = [
        (1, '張三', 'zhang@example.com'),
        (2, '李四', 'li@example.com'),
        (3, '王五', 'wang@example.com')
    ]
    
    products_data = [
        (1, 'iPhone 15', 999.99, '電子產品', '最新款智能手機'),
        (2, 'MacBook Pro', 1299.99, '電子產品', '專業筆記本電腦'),
        (3, '咖啡機', 199.99, '家電', '全自動咖啡機'),
        (4, '書籍：Python編程', 29.99, '圖書', 'Python編程入門教程')
    ]
    
    orders_data = [
        (1, 1, 1, 1),
        (2, 1, 3, 1),
        (3, 2, 2, 1),
        (4, 3, 4, 2)
    ]
    
    cursor.executemany('INSERT OR REPLACE INTO users (user_id, name, email) VALUES (?, ?, ?)', users_data)
    cursor.executemany('INSERT OR REPLACE INTO products (product_id, name, price, category, description) VALUES (?, ?, ?, ?, ?)', products_data)
    cursor.executemany('INSERT OR REPLACE INTO orders (order_id, user_id, product_id, quantity) VALUES (?, ?, ?, ?)', orders_data)
    
    conn.commit()
    conn.close()
    print("示例數據庫創建完成！")

# 創建數據庫
create_sample_database()

# 驗證數據
def verify_database():
    conn = sqlite3.connect('sample_ecommerce.db')
    
    print("用戶表:")
    users_df = pd.read_sql_query('SELECT * FROM users', conn)
    print(users_df)
    
    print("\n產品表:")
    products_df = pd.read_sql_query('SELECT * FROM products', conn)
    print(products_df)
    
    print("\n訂單表:")
    orders_df = pd.read_sql_query('SELECT * FROM orders', conn)
    print(orders_df)
    
    conn.close()

verify_database()

### 準備訓練數據集

In [ ]:
# 準備訓練數據集 - 使用專業數據集和自製數據集結合

print("🔍 獲取專業 Text2SQL 數據集...")

# 方案1: 使用 Hugging Face 專業數據集
def load_professional_datasets():
    """加載專業 Text2SQL 數據集"""
    professional_data = []
    
    try:
        # 嘗試使用 datasets 庫加載專業數據集
        from datasets import load_dataset
        
        print("📦 嘗試加載 Hugging Face 數據集...")
        
        # 加載一些知名的 Text2SQL 數據集
        dataset_configs = [
            ("gretelai/synthetic_text_to_sql", "部分合成數據集"),
            ("NumbersStation/NSText2SQL", "專業基準數據集"),
            ("spider", "學術標準數據集")
        ]
        
        for dataset_name, description in dataset_configs:
            try:
                print(f"  • 嘗試加載: {dataset_name}")
                dataset = load_dataset(dataset_name, split="train[:100]")  # 只加載前100條
                
                for item in dataset:
                    # 統一數據格式
                    if 'question' in item and 'query' in item:
                        professional_data.append({
                            "natural_language": item['question'],
                            "sql": item['query'],
                            "explanation": f"來自{description}",
                            "source": dataset_name
                        })
                
                print(f"    ✓ 成功加載 {len([d for d in professional_data if d['source'] == dataset_name])} 條數據")
                break  # 成功加載一個數據集即可
                
            except Exception as e:
                print(f"    ⚠️ {dataset_name} 加載失敗: {str(e)[:50]}...")
                continue
                
    except ImportError:
        print("⚠️ datasets 庫不可用，使用本地數據集")
    
    return professional_data

# 嘗試加載專業數據集
professional_data = load_professional_datasets()

# 方案2: 自製基礎數據集（用於學習和備用）
print("\n📝 準備基礎訓練數據集...")
basic_training_data = [
    {
        "natural_language": "顯示所有用戶的信息",
        "sql": "SELECT * FROM users;",
        "explanation": "查詢用戶表中的所有記錄",
        "source": "basic_examples"
    },
    {
        "natural_language": "找出價格超過500元的產品",
        "sql": "SELECT * FROM products WHERE price > 500;",
        "explanation": "使用WHERE子句過濾價格大於500的產品",
        "source": "basic_examples"
    },
    {
        "natural_language": "統計每個用戶的訂單數量",
        "sql": "SELECT u.name, COUNT(o.order_id) as order_count FROM users u LEFT JOIN orders o ON u.user_id = o.user_id GROUP BY u.user_id, u.name;",
        "explanation": "使用JOIN和GROUP BY統計每個用戶的訂單數量",
        "source": "basic_examples"
    },
    {
        "natural_language": "查找最貴的產品",
        "sql": "SELECT * FROM products ORDER BY price DESC LIMIT 1;",
        "explanation": "使用ORDER BY和LIMIT找到價格最高的產品",
        "source": "basic_examples"
    },
    {
        "natural_language": "顯示用戶張三的所有訂單",
        "sql": "SELECT o.*, p.name as product_name FROM orders o JOIN users u ON o.user_id = u.user_id JOIN products p ON o.product_id = p.product_id WHERE u.name = '張三';",
        "explanation": "使用多表JOIN查詢特定用戶的訂單信息",
        "source": "basic_examples"
    },
    {
        "natural_language": "計算每個類別的產品平均價格",
        "sql": "SELECT category, AVG(price) as avg_price FROM products GROUP BY category;",
        "explanation": "使用GROUP BY和AVG函數計算各類別的平均價格",
        "source": "basic_examples"
    },
    {
        "natural_language": "找出沒有下過訂單的用戶",
        "sql": "SELECT u.* FROM users u LEFT JOIN orders o ON u.user_id = o.user_id WHERE o.user_id IS NULL;",
        "explanation": "使用LEFT JOIN和IS NULL找出沒有訂單的用戶",
        "source": "basic_examples"
    },
    {
        "natural_language": "顯示銷量最高的產品",
        "sql": "SELECT p.name, SUM(o.quantity) as total_sold FROM products p JOIN orders o ON p.product_id = o.product_id GROUP BY p.product_id, p.name ORDER BY total_sold DESC LIMIT 1;",
        "explanation": "統計產品銷量並找出銷量最高的產品",
        "source": "basic_examples"
    }
]

# 方案3: 整合數據集
print("\n🔄 整合數據集...")
if professional_data:
    training_data = professional_data + basic_training_data
    print(f"✓ 使用混合數據集: {len(professional_data)} 條專業數據 + {len(basic_training_data)} 條基礎數據")
else:
    training_data = basic_training_data
    print(f"✓ 使用基礎數據集: {len(basic_training_data)} 條自製數據")

# 數據集統計
sources = {}
for item in training_data:
    source = item.get('source', 'unknown')
    sources[source] = sources.get(source, 0) + 1

print(f"\n📊 數據集統計 (總計 {len(training_data)} 條):")
for source, count in sources.items():
    print(f"  • {source}: {count} 條")

# 保存完整數據集
with open('training_data.json', 'w', encoding='utf-8') as f:
    json.dump(training_data, f, ensure_ascii=False, indent=2)

print(f"\n💾 數據集已保存到 training_data.json")

# 顯示數據集示例
print(f"\n📋 數據集示例:")
for i, item in enumerate(training_data[:3]):
    print(f"\n樣本 {i+1} ({item.get('source', 'unknown')}):")
    print(f"  自然語言: {item['natural_language']}")
    print(f"  SQL: {item['sql']}")
    print(f"  說明: {item['explanation']}")

print(f"\n💡 數據集來源說明:")
print(f"  • 專業數據集: 來自 Hugging Face，質量高，覆蓋面廣")
print(f"  • 基礎數據集: 針對示例數據庫設計，便於學習理解")
print(f"  • 混合策略: 結合專業性和針對性，提升RAG效果")

## 5. 向量化與索引

### 使用句子嵌入模型

In [ ]:
# 初始化句子嵌入模型（增強版，支援備用方案）
class TextEmbedder:
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        """初始化文本嵌入器"""
        self.model = None
        self.use_simple_encoding = False
        
        if SENTENCE_TRANSFORMERS_AVAILABLE:
            try:
                self.model = SentenceTransformer(model_name)
                print(f"✓ 成功加載模型: {model_name}")
            except Exception as e:
                print(f"⚠️ 模型加載失敗: {e}, 使用備用方案")
                self.use_simple_encoding = True
        else:
            print("⚠️ sentence_transformers 不可用，使用簡單編碼方案")
            self.use_simple_encoding = True
    
    def encode(self, texts: List[str]) -> np.ndarray:
        """將文本編碼為向量"""
        if self.model and not self.use_simple_encoding:
            try:
                return self.model.encode(texts)
            except Exception as e:
                print(f"⚠️ 模型編碼失敗: {e}, 切換到備用方案")
                self.use_simple_encoding = True
                return self._simple_encode(texts)
        else:
            return self._simple_encode(texts)
    
    def _simple_encode(self, texts: List[str]) -> np.ndarray:
        """簡單的文本編碼備用方案（基於TF-IDF概念）"""
        # 構建詞彙表
        all_words = set()
        for text in texts:
            words = text.lower().replace('，', ' ').replace('。', ' ').split()
            all_words.update(words)
        
        vocab = {word: idx for idx, word in enumerate(sorted(all_words))}
        
        # 創建向量
        vectors = []
        for text in texts:
            vector = np.zeros(len(vocab))
            words = text.lower().replace('，', ' ').replace('。', ' ').split()
            
            # 簡單的詞頻統計
            word_count = {}
            for word in words:
                word_count[word] = word_count.get(word, 0) + 1
            
            # 填充向量
            for word, count in word_count.items():
                if word in vocab:
                    vector[vocab[word]] = count
            
            # 正規化
            if np.linalg.norm(vector) > 0:
                vector = vector / np.linalg.norm(vector)
            
            vectors.append(vector)
        
        return np.array(vectors)

# 初始化嵌入器
embedder = TextEmbedder()

# 測試嵌入功能
test_texts = [
    "查詢所有用戶信息",
    "顯示產品詳細信息",
    "統計訂單數量"
]

embeddings = embedder.encode(test_texts)
print(f"\n📊 文本嵌入測試:")
print(f"  • 嵌入維度: {embeddings.shape}")
print(f"  • 編碼方式: {'神經網絡模型' if not embedder.use_simple_encoding else '簡單TF-IDF'}")
print(f"  • 示例向量前5維: {embeddings[0][:5]}")

# 測試相似度
if len(embeddings) >= 2:
    similarity = np.dot(embeddings[0], embeddings[1])
    print(f"  • 前兩個文本相似度: {similarity:.3f}")

### 構建向量索引

In [ ]:
# 構建向量索引（增強版，支援FAISS備用方案）
class VectorIndex:
    def __init__(self, dimension: int):
        """初始化向量索引"""
        self.dimension = dimension
        self.texts = []  # 存儲原始文本
        self.metadata = []  # 存儲元數據
        self.vectors = []  # 備用方案：存儲所有向量
        self.use_faiss = FAISS_AVAILABLE
        
        if self.use_faiss:
            try:
                self.index = faiss.IndexFlatL2(dimension)  # L2距離索引
                print(f"✓ 使用 FAISS 索引，維度: {dimension}")
            except Exception as e:
                print(f"⚠️ FAISS 初始化失敗: {e}, 使用備用索引")
                self.use_faiss = False
        
        if not self.use_faiss:
            print(f"✓ 使用簡單向量索引，維度: {dimension}")
    
    def add_vectors(self, vectors: np.ndarray, texts: List[str], metadata: List[Dict]):
        """添加向量到索引"""
        if self.use_faiss:
            try:
                self.index.add(vectors.astype('float32'))
            except Exception as e:
                print(f"⚠️ FAISS 添加向量失敗: {e}, 切換到備用方案")
                self.use_faiss = False
                self.vectors = vectors.tolist()  # 轉換為列表存儲
        
        if not self.use_faiss:
            self.vectors.extend(vectors.tolist())
        
        self.texts.extend(texts)
        self.metadata.extend(metadata)
        print(f"✓ 已添加 {len(vectors)} 個向量到索引")
    
    def search(self, query_vector: np.ndarray, k: int = 5) -> List[Dict]:
        """搜索最相似的向量"""
        if self.use_faiss:
            return self._faiss_search(query_vector, k)
        else:
            return self._simple_search(query_vector, k)
    
    def _faiss_search(self, query_vector: np.ndarray, k: int) -> List[Dict]:
        """使用 FAISS 搜索"""
        try:
            query_vector = query_vector.reshape(1, -1).astype('float32')
            distances, indices = self.index.search(query_vector, k)
            
            results = []
            for i, (distance, idx) in enumerate(zip(distances[0], indices[0])):
                if idx < len(self.texts):
                    results.append({
                        'text': self.texts[idx],
                        'metadata': self.metadata[idx],
                        'distance': float(distance),
                        'rank': i + 1
                    })
            return results
        except Exception as e:
            print(f"⚠️ FAISS 搜索失敗: {e}, 切換到備用搜索")
            self.use_faiss = False
            return self._simple_search(query_vector, k)
    
    def _simple_search(self, query_vector: np.ndarray, k: int) -> List[Dict]:
        """簡單的向量搜索（餘弦相似度）"""
        if not self.vectors:
            return []
        
        # 計算與所有向量的相似度
        similarities = []
        query_norm = np.linalg.norm(query_vector)
        
        for i, stored_vector in enumerate(self.vectors):
            stored_vector = np.array(stored_vector)
            stored_norm = np.linalg.norm(stored_vector)
            
            if query_norm > 0 and stored_norm > 0:
                # 餘弦相似度
                similarity = np.dot(query_vector, stored_vector) / (query_norm * stored_norm)
                # 轉換為距離（距離越小越相似）
                distance = 1 - similarity
            else:
                distance = float('inf')
            
            similarities.append((distance, i))
        
        # 排序並返回前k個
        similarities.sort(key=lambda x: x[0])
        
        results = []
        for rank, (distance, idx) in enumerate(similarities[:k]):
            if idx < len(self.texts):
                results.append({
                    'text': self.texts[idx],
                    'metadata': self.metadata[idx],
                    'distance': float(distance),
                    'rank': rank + 1
                })
        
        return results
    
    def get_stats(self) -> Dict:
        """獲取索引統計信息"""
        total_vectors = len(self.vectors) if not self.use_faiss else (self.index.ntotal if hasattr(self, 'index') else 0)
        return {
            'total_vectors': total_vectors,
            'dimension': self.dimension,
            'index_type': 'FAISS-FlatL2' if self.use_faiss else 'Simple-Cosine'
        }

# 為訓練數據創建向量索引
natural_language_queries = [item['natural_language'] for item in training_data]
query_embeddings = embedder.encode(natural_language_queries)

# 初始化向量索引
vector_index = VectorIndex(query_embeddings.shape[1])

# 添加向量到索引
vector_index.add_vectors(
    query_embeddings,
    natural_language_queries,
    training_data
)

print(f"\n📊 向量索引統計:")
stats = vector_index.get_stats()
for key, value in stats.items():
    print(f"  • {key}: {value}")

# 測試搜索功能
test_query = "查詢用戶信息"
test_vector = embedder.encode([test_query])[0]
search_results = vector_index.search(test_vector, k=3)

print(f"\n🔍 搜索測試: '{test_query}'")
for result in search_results:
    print(f"  • 匹配: {result['text']} (距離: {result['distance']:.3f})")

## 6. 檢索系統

### 實現語義檢索

In [ ]:
# 實現檢索系統
class SemanticRetriever:
    def __init__(self, embedder: TextEmbedder, vector_index: VectorIndex, schema: DatabaseSchema):
        """初始化語義檢索器"""
        self.embedder = embedder
        self.vector_index = vector_index
        self.schema = schema
    
    def retrieve(self, query: str, k: int = 3) -> List[Dict]:
        """檢索相關的SQL示例"""
        # 1. 將查詢編碼為向量
        query_vector = self.embedder.encode([query])[0]
        
        # 2. 在向量索引中搜索
        similar_examples = self.vector_index.search(query_vector, k)
        
        # 3. 添加數據庫模式信息
        schema_info = self.schema.get_schema_info()
        
        # 4. 構建檢索結果
        retrieval_results = {
            'query': query,
            'schema_info': schema_info,
            'similar_examples': similar_examples,
            'retrieval_score': self._calculate_retrieval_score(similar_examples)
        }
        
        return retrieval_results
    
    def _calculate_retrieval_score(self, examples: List[Dict]) -> float:
        """計算檢索質量分數"""
        if not examples:
            return 0.0
        
        # 基於距離計算分數（距離越小，分數越高）
        distances = [ex['distance'] for ex in examples]
        avg_distance = sum(distances) / len(distances)
        score = max(0, 1 - avg_distance / 10)  # 簡單的評分公式
        return score
    
    def explain_retrieval(self, results: Dict) -> str:
        """解釋檢索過程"""
        explanation = f"\n檢索結果解釋：\n"
        explanation += f"查詢: {results['query']}\n"
        explanation += f"檢索分數: {results['retrieval_score']:.3f}\n"
        explanation += f"找到 {len(results['similar_examples'])} 個相似示例:\n"
        
        for i, example in enumerate(results['similar_examples']):
            explanation += f"\n{i+1}. 相似度: {1-example['distance']:.3f}\n"
            explanation += f"   查詢: {example['text']}\n"
            explanation += f"   SQL: {example['metadata']['sql']}\n"
        
        return explanation

# 初始化檢索器
retriever = SemanticRetriever(embedder, vector_index, schema)

# 測試檢索功能
test_query = "我要查看所有產品的詳細信息"
retrieval_results = retriever.retrieve(test_query)

print("=== 檢索測試 ===")
print(retriever.explain_retrieval(retrieval_results))

## 7. SQL 生成

### 基於檢索結果生成SQL

In [ ]:
# SQL生成器 (2025年增強版 - 支援現代模型)
import re
from datetime import datetime

class SQLGenerator:
    def __init__(self, schema: DatabaseSchema, model_approach="rag"):
        """初始化SQL生成器
        
        Args:
            schema: 數據庫模式
            model_approach: 'rag' (檢索增強) 或 'llm' (大語言模型)
        """
        self.schema = schema
        self.model_approach = model_approach
        self.templates = self._load_sql_templates()
        
        # 2025年推薦的模型配置
        self.recommended_models = {
            "kaggle_friendly": {
                "name": "Salesforce/codet5p-220m",
                "size": "220M",
                "memory": "~1GB",
                "description": "最適合Kaggle，專為代碼生成設計"
            },
            "balanced": {
                "name": "TinyLlama/TinyLlama-1.1B-Chat-v1.0", 
                "size": "1.1B",
                "memory": "~3GB",
                "description": "平衡效能與資源，支援QLoRA"
            },
            "production": {
                "name": "mistralai/Mistral-7B-Instruct-v0.3",
                "size": "7B", 
                "memory": "~14GB",
                "description": "生產級別，需要較多資源"
            }
        }
        
        # 嘗試載入現代LLM（如果可用）
        self.llm_available = self._try_load_llm()
        
        print(f"✓ SQL生成器初始化完成")
        print(f"  • 生成方法: {model_approach.upper()}")
        print(f"  • LLM可用性: {'是' if self.llm_available else '否 (使用模板方法)'}")
    
    def _try_load_llm(self):
        """嘗試載入LLM模型"""
        if self.model_approach == "rag":
            return False  # RAG方法不需要載入LLM
        
        try:
            if TRANSFORMERS_AVAILABLE:
                # 這裡可以實際載入模型，但為了演示先跳過
                print("  💡 提示: 要使用LLM生成，請安裝並載入推薦模型")
                return True
            return False
        except Exception as e:
            print(f"  ⚠️ LLM載入失敗: {e}")
            return False
    
    def get_model_recommendations(self) -> str:
        """獲取2025年模型推薦"""
        recommendations = "\n🏆 2025年TextSQL模型推薦:\n"
        recommendations += "=" * 50 + "\n"
        
        for category, info in self.recommended_models.items():
            recommendations += f"\n📱 {category.upper()}:\n"
            recommendations += f"  • 模型: {info['name']}\n"
            recommendations += f"  • 大小: {info['size']}\n" 
            recommendations += f"  • 記憶體: {info['memory']}\n"
            recommendations += f"  • 描述: {info['description']}\n"
        
        recommendations += f"\n💡 微調建議:\n"
        recommendations += f"  • 首選: QLoRA (4-bit量化 + LoRA)\n"
        recommendations += f"  • 備選: LoRA (8-bit量化)\n"
        recommendations += f"  • 高端: Full Fine-tuning\n"
        
        recommendations += f"\n🤔 是否需要微調?\n"
        recommendations += f"  • RAG方法: ❌ 無需微調，成本低\n"
        recommendations += f"  • 特定領域: ✅ 微調提升精度\n"
        recommendations += f"  • 通用場景: ❌ 預訓練模型已足夠\n"
        
        return recommendations
    
    def _load_sql_templates(self) -> Dict[str, str]:
        """加載SQL模板"""
        return {
            'select_all': "SELECT * FROM {table};",
            'select_where': "SELECT * FROM {table} WHERE {condition};",
            'count': "SELECT COUNT(*) FROM {table};",
            'join': "SELECT {columns} FROM {table1} t1 JOIN {table2} t2 ON {join_condition};",
            'group_by': "SELECT {columns}, {aggregate} FROM {table} GROUP BY {group_columns};",
            'order_by': "SELECT * FROM {table} ORDER BY {column} {direction};",
            'avg': "SELECT AVG({column}) as avg_{column} FROM {table};",
            'sum': "SELECT SUM({column}) as total_{column} FROM {table};",
            'max': "SELECT MAX({column}) as max_{column} FROM {table};",
            'min': "SELECT MIN({column}) as min_{column} FROM {table};"
        }
    
    def generate_sql(self, query: str, retrieval_results: Dict) -> Dict:
        """基於檢索結果生成SQL"""
        timestamp = datetime.now().isoformat()
        
        # 1. 分析查詢意圖
        intent = self._analyze_intent(query)
        
        # 2. 從檢索結果中提取最佳匹配
        best_match = retrieval_results['similar_examples'][0] if retrieval_results['similar_examples'] else None
        
        # 3. 根據方法生成SQL
        if self.model_approach == "llm" and self.llm_available:
            generated_sql, method = self._llm_generation(query, retrieval_results, intent)
        else:
            generated_sql, method = self._rag_generation(query, retrieval_results, intent, best_match)
        
        # 4. 驗證SQL
        is_valid, validation_error = self._validate_sql(generated_sql)
        
        # 5. 後處理
        generated_sql = self._post_process_sql(generated_sql)
        
        return {
            'query': query,
            'generated_sql': generated_sql,
            'method': method,
            'intent': intent,
            'is_valid': is_valid,
            'validation_error': validation_error,
            'best_match': best_match,
            'timestamp': timestamp,
            'model_approach': self.model_approach
        }
    
    def _llm_generation(self, query: str, retrieval_results: Dict, intent: Dict) -> tuple:
        """使用LLM生成SQL (2025年方法)"""
        # 構建提示詞
        prompt = self._build_llm_prompt(query, retrieval_results, intent)
        
        # 這裡應該調用實際的LLM API
        # generated_sql = llm_model.generate(prompt)
        
        # 為演示目的，返回基礎生成結果
        generated_sql, _ = self._rag_generation(query, retrieval_results, intent, None)
        return generated_sql, "llm_enhanced"
    
    def _rag_generation(self, query: str, retrieval_results: Dict, intent: Dict, best_match) -> tuple:
        """使用RAG方法生成SQL"""
        if best_match and best_match['distance'] < 0.5:  # 高相似度
            generated_sql = self._adapt_sql(best_match['metadata']['sql'], query, intent)
            method = 'retrieval_based'
        else:
            generated_sql = self._template_based_generation(query, intent)
            method = 'template_based'
        
        return generated_sql, method
    
    def _build_llm_prompt(self, query: str, retrieval_results: Dict, intent: Dict) -> str:
        """構建LLM提示詞 (2025年最佳實踐)"""
        prompt = f"""你是一個專業的SQL生成助手。根據以下信息生成準確的SQL查詢。

數據庫架構:
{self.schema.get_schema_info()}

用戶查詢: {query}

相似示例:
"""
        
        for i, example in enumerate(retrieval_results['similar_examples'][:2]):
            prompt += f"{i+1}. 問題: {example['text']}\n"
            prompt += f"   SQL: {example['metadata']['sql']}\n\n"
        
        prompt += """請生成對應的SQL查詢。要求:
1. 語法正確
2. 符合數據庫架構
3. 回答用戶問題
4. 只返回SQL語句，不要額外解釋

SQL:"""
        
        return prompt
    
    def _analyze_intent(self, query: str) -> Dict:
        """增強的查詢意圖分析"""
        query_lower = query.lower()
        
        intent = {
            'action': 'select',
            'tables': [],
            'conditions': [],
            'aggregation': None,
            'sorting': None,
            'limit': None,
            'join_type': None
        }
        
        # 識別表名
        for table in self.schema.tables.keys():
            if table in query_lower or self._table_synonyms(table, query_lower):
                intent['tables'].append(table)
        
        # 識別聚合操作
        aggregation_keywords = {
            '統計': 'count', '計算': 'count', '總和': 'sum', 
            '平均': 'avg', '最大': 'max', '最小': 'min',
            '數量': 'count'
        }
        for keyword, agg_type in aggregation_keywords.items():
            if keyword in query_lower:
                intent['aggregation'] = agg_type
                break
        
        # 識別排序
        if any(word in query_lower for word in ['最高', '最大', '最貴', '降序']):
            intent['sorting'] = 'desc'
        elif any(word in query_lower for word in ['最低', '最小', '最便宜', '升序']):
            intent['sorting'] = 'asc'
        
        # 識別限制
        if any(word in query_lower for word in ['前', '最', '第一']):
            intent['limit'] = 1
        
        return intent
    
    def _table_synonyms(self, table: str, query: str) -> bool:
        """擴展的表名同義詞"""
        synonyms = {
            'users': ['用戶', '使用者', '會員', '客戶', '用户'],
            'products': ['產品', '商品', '物品', '貨品', '产品'],
            'orders': ['訂單', '訂購', '購買', '交易', '订单']
        }
        
        if table in synonyms:
            return any(syn in query for syn in synonyms[table])
        return False
    
    def _adapt_sql(self, base_sql: str, query: str, intent: Dict) -> str:
        """智能SQL適配"""
        adapted_sql = base_sql
        
        # 根據意圖調整SQL
        if intent['limit'] and 'LIMIT' not in adapted_sql.upper():
            adapted_sql = adapted_sql.rstrip(';') + f" LIMIT {intent['limit']};"
        
        return adapted_sql
    
    def _template_based_generation(self, query: str, intent: Dict) -> str:
        """增強的模板生成"""
        if not intent['tables']:
            return "SELECT 'No table identified' as error;"
        
        table = intent['tables'][0]
        
        # 聚合查詢
        if intent['aggregation']:
            agg_func = intent['aggregation'].upper()
            if agg_func == 'COUNT':
                sql = f"SELECT COUNT(*) as total_count FROM {table};"
            else:
                numeric_columns = self._get_numeric_columns(table)
                if numeric_columns:
                    column = numeric_columns[0]
                    sql = f"SELECT {agg_func}({column}) as {agg_func.lower()}_{column} FROM {table};"
                else:
                    sql = f"SELECT COUNT(*) as total_count FROM {table};"
        
        # 排序查詢
        elif intent['sorting']:
            numeric_columns = self._get_numeric_columns(table)
            if numeric_columns:
                column = numeric_columns[0]
                direction = intent['sorting'].upper()
                sql = f"SELECT * FROM {table} ORDER BY {column} {direction};"
                if intent['limit']:
                    sql = sql.rstrip(';') + f" LIMIT {intent['limit']};"
            else:
                sql = f"SELECT * FROM {table};"
        
        # 基本查詢
        else:
            sql = f"SELECT * FROM {table};"
            if intent['limit']:
                sql = sql.rstrip(';') + f" LIMIT {intent['limit']};"
        
        return sql
    
    def _get_numeric_columns(self, table: str) -> List[str]:
        """獲取數值列"""
        if table not in self.schema.tables:
            return []
        
        numeric_types = ['INT', 'DECIMAL', 'FLOAT', 'DOUBLE', 'NUMERIC']
        numeric_columns = []
        
        for column, column_type in self.schema.tables[table].items():
            if any(num_type in column_type.upper() for num_type in numeric_types):
                numeric_columns.append(column)
        
        return numeric_columns
    
    def _post_process_sql(self, sql: str) -> str:
        """SQL後處理"""
        # 確保以分號結尾
        sql = sql.strip()
        if not sql.endswith(';'):
            sql += ';'
        
        # 格式化
        sql = re.sub(r'\s+', ' ', sql)  # 統一空格
        
        return sql
    
    def _validate_sql(self, sql: str) -> Tuple[bool, str]:
        """增強的SQL驗證"""
        try:
            parsed = sqlparse.parse(sql)[0]
            if parsed.tokens:
                # 基本語法檢查
                sql_upper = sql.upper()
                if not any(keyword in sql_upper for keyword in ['SELECT', 'INSERT', 'UPDATE', 'DELETE']):
                    return False, "不包含有效的SQL關鍵字"
                
                return True, ""
            else:
                return False, "空的SQL語句"
        except Exception as e:
            return False, str(e)

# 初始化增強版SQL生成器
sql_generator = SQLGenerator(schema, model_approach="rag")

# 顯示模型推薦
print(sql_generator.get_model_recommendations())

# 測試增強版SQL生成
test_queries = [
    "顯示所有產品信息",
    "找出最貴的產品", 
    "統計用戶數量",
    "計算產品平均價格",
    "顯示前3個最貴的產品"
]

print(f"\n🧪 增強版SQL生成測試:")
print("=" * 50)

for query in test_queries:
    retrieval_results = retriever.retrieve(query)
    generation_results = sql_generator.generate_sql(query, retrieval_results)
    
    print(f"\n🔍 查詢: {query}")
    print(f"📝 生成SQL: {generation_results['generated_sql']}")
    print(f"🔧 方法: {generation_results['method']}")
    print(f"🎯 意圖: {generation_results['intent']}")
    print(f"✅ 有效: {generation_results['is_valid']}")
    
    if generation_results['validation_error']:
        print(f"❌ 錯誤: {generation_results['validation_error']}")

## 8. 完整流水線

### 整合所有組件

In [ ]:
# 完整的TextSQL RAG流水線
class TextSQLRAGPipeline:
    def __init__(self, 
                 embedder: TextEmbedder,
                 vector_index: VectorIndex,
                 schema: DatabaseSchema,
                 retriever: SemanticRetriever,
                 sql_generator: SQLGenerator):
        """初始化完整的RAG流水線"""
        self.embedder = embedder
        self.vector_index = vector_index
        self.schema = schema
        self.retriever = retriever
        self.sql_generator = sql_generator
        self.query_history = []
    
    def process_query(self, natural_language_query: str, execute_sql: bool = False) -> Dict:
        """處理自然語言查詢的完整流程"""
        print(f"\n處理查詢: {natural_language_query}")
        
        # 第1步：檢索相關示例
        print("第1步：檢索相關示例...")
        retrieval_results = self.retriever.retrieve(natural_language_query)
        
        # 第2步：生成SQL
        print("第2步：生成SQL...")
        generation_results = self.sql_generator.generate_sql(natural_language_query, retrieval_results)
        
        # 第3步：執行SQL（可選）
        execution_results = None
        if execute_sql and generation_results['is_valid']:
            print("第3步：執行SQL...")
            execution_results = self._execute_sql(generation_results['generated_sql'])
        
        # 第4步：構建完整結果
        complete_results = {
            'natural_language_query': natural_language_query,
            'retrieval_results': retrieval_results,
            'generation_results': generation_results,
            'execution_results': execution_results,
            'pipeline_success': generation_results['is_valid'],
            'timestamp': pd.Timestamp.now().isoformat()
        }
        
        # 第5步：記錄查詢歷史
        self.query_history.append(complete_results)
        
        return complete_results
    
    def _execute_sql(self, sql: str) -> Dict:
        """執行SQL查詢"""
        try:
            conn = sqlite3.connect('sample_ecommerce.db')
            
            # 執行查詢
            result_df = pd.read_sql_query(sql, conn)
            
            conn.close()
            
            return {
                'success': True,
                'data': result_df.to_dict('records'),
                'row_count': len(result_df),
                'columns': list(result_df.columns),
                'error': None
            }
        
        except Exception as e:
            return {
                'success': False,
                'data': None,
                'row_count': 0,
                'columns': [],
                'error': str(e)
            }
    
    def explain_results(self, results: Dict) -> str:
        """解釋處理結果"""
        explanation = f"\n{'='*50}\n"
        explanation += f"查詢: {results['natural_language_query']}\n"
        explanation += f"{'='*50}\n"
        
        # 檢索階段
        explanation += "\n🔍 檢索階段:\n"
        retrieval = results['retrieval_results']
        explanation += f"找到 {len(retrieval['similar_examples'])} 個相似示例\n"
        explanation += f"檢索分數: {retrieval['retrieval_score']:.3f}\n"
        
        if retrieval['similar_examples']:
            best_match = retrieval['similar_examples'][0]
            explanation += f"最佳匹配: {best_match['text']} (相似度: {1-best_match['distance']:.3f})\n"
        
        # 生成階段
        explanation += "\n⚙️ 生成階段:\n"
        generation = results['generation_results']
        explanation += f"生成方法: {generation['method']}\n"
        explanation += f"生成的SQL: {generation['generated_sql']}\n"
        explanation += f"SQL有效性: {generation['is_valid']}\n"
        
        # 執行階段
        if results['execution_results']:
            explanation += "\n🚀 執行階段:\n"
            execution = results['execution_results']
            if execution['success']:
                explanation += f"執行成功，返回 {execution['row_count']} 行數據\n"
                if execution['data']:
                    explanation += "前幾行數據:\n"
                    for i, row in enumerate(execution['data'][:3]):
                        explanation += f"  {i+1}: {row}\n"
            else:
                explanation += f"執行失敗: {execution['error']}\n"
        
        explanation += f"\n✅ 流水線成功: {results['pipeline_success']}\n"
        
        return explanation
    
    def get_pipeline_stats(self) -> Dict:
        """獲取流水線統計信息"""
        if not self.query_history:
            return {'total_queries': 0}
        
        successful_queries = sum(1 for q in self.query_history if q['pipeline_success'])
        
        return {
            'total_queries': len(self.query_history),
            'successful_queries': successful_queries,
            'success_rate': successful_queries / len(self.query_history),
            'avg_retrieval_score': np.mean([q['retrieval_results']['retrieval_score'] 
                                          for q in self.query_history])
        }

# 初始化完整流水線
pipeline = TextSQLRAGPipeline(
    embedder=embedder,
    vector_index=vector_index,
    schema=schema,
    retriever=retriever,
    sql_generator=sql_generator
)

print("✅ TextSQL RAG 流水線初始化完成！")

### 流水線演示

In [ ]:
# 流水線演示
demo_queries = [
    "顯示所有用戶的基本信息",
    "查找價格最高的產品",
    "統計每個類別有多少產品",
    "顯示張三購買的所有商品",
    "找出還沒有下過訂單的用戶"
]

print("🚀 開始流水線演示\n")

for i, query in enumerate(demo_queries, 1):
    print(f"\n{'='*60}")
    print(f"演示 {i}/{len(demo_queries)}")
    print(f"{'='*60}")
    
    # 處理查詢
    results = pipeline.process_query(query, execute_sql=True)
    
    # 顯示結果
    print(pipeline.explain_results(results))
    
    # 暫停一下讓輸出更清晰
    import time
    time.sleep(1)

# 顯示整體統計
print("\n" + "="*60)
print("流水線統計")
print("="*60)
stats = pipeline.get_pipeline_stats()
for key, value in stats.items():
    print(f"{key}: {value}")

## 9. 評估與優化

### 評估指標

In [ ]:
# 評估系統
class RAGEvaluator:
    def __init__(self, pipeline: TextSQLRAGPipeline):
        """初始化評估器"""
        self.pipeline = pipeline
    
    def evaluate_retrieval(self, test_cases: List[Dict]) -> Dict:
        """評估檢索性能"""
        retrieval_scores = []
        precision_scores = []
        
        for test_case in test_cases:
            query = test_case['natural_language']
            expected_sql = test_case['sql']
            
            # 檢索相關示例
            retrieval_results = self.pipeline.retriever.retrieve(query)
            
            # 計算檢索分數
            retrieval_scores.append(retrieval_results['retrieval_score'])
            
            # 計算精確度（檢索結果中是否包含正確答案）
            precision = self._calculate_precision(retrieval_results, expected_sql)
            precision_scores.append(precision)
        
        return {
            'avg_retrieval_score': np.mean(retrieval_scores),
            'avg_precision': np.mean(precision_scores),
            'retrieval_scores': retrieval_scores,
            'precision_scores': precision_scores
        }
    
    def evaluate_generation(self, test_cases: List[Dict]) -> Dict:
        """評估SQL生成性能"""
        exact_match_scores = []
        syntax_valid_scores = []
        semantic_similarity_scores = []
        
        for test_case in test_cases:
            query = test_case['natural_language']
            expected_sql = test_case['sql']
            
            # 生成SQL
            results = self.pipeline.process_query(query, execute_sql=False)
            generated_sql = results['generation_results']['generated_sql']
            
            # 精確匹配
            exact_match = self._normalize_sql(generated_sql) == self._normalize_sql(expected_sql)
            exact_match_scores.append(exact_match)
            
            # 語法有效性
            syntax_valid = results['generation_results']['is_valid']
            syntax_valid_scores.append(syntax_valid)
            
            # 語義相似度（簡化實現）
            semantic_sim = self._calculate_semantic_similarity(generated_sql, expected_sql)
            semantic_similarity_scores.append(semantic_sim)
        
        return {
            'exact_match_rate': np.mean(exact_match_scores),
            'syntax_valid_rate': np.mean(syntax_valid_scores),
            'avg_semantic_similarity': np.mean(semantic_similarity_scores),
            'exact_match_scores': exact_match_scores,
            'syntax_valid_scores': syntax_valid_scores,
            'semantic_similarity_scores': semantic_similarity_scores
        }
    
    def evaluate_end_to_end(self, test_cases: List[Dict]) -> Dict:
        """端到端評估"""
        execution_success_scores = []
        result_accuracy_scores = []
        
        for test_case in test_cases:
            query = test_case['natural_language']
            expected_sql = test_case['sql']
            
            # 執行完整流水線
            results = self.pipeline.process_query(query, execute_sql=True)
            
            # 執行成功率
            execution_success = (
                results['execution_results'] is not None and 
                results['execution_results']['success']
            )
            execution_success_scores.append(execution_success)
            
            # 結果准確性（通過執行期望SQL進行比較）
            if execution_success:
                result_accuracy = self._compare_execution_results(
                    results['generation_results']['generated_sql'],
                    expected_sql
                )
                result_accuracy_scores.append(result_accuracy)
            else:
                result_accuracy_scores.append(0.0)
        
        return {
            'execution_success_rate': np.mean(execution_success_scores),
            'result_accuracy_rate': np.mean(result_accuracy_scores),
            'execution_success_scores': execution_success_scores,
            'result_accuracy_scores': result_accuracy_scores
        }
    
    def _calculate_precision(self, retrieval_results: Dict, expected_sql: str) -> float:
        """計算檢索精確度"""
        similar_examples = retrieval_results['similar_examples']
        if not similar_examples:
            return 0.0
        
        # 檢查是否有檢索結果與期望SQL相似
        for example in similar_examples:
            if self._sql_similarity(example['metadata']['sql'], expected_sql) > 0.8:
                return 1.0
        
        return 0.0
    
    def _normalize_sql(self, sql: str) -> str:
        """標準化SQL字符串"""
        # 移除多餘空格，轉換為小寫
        return ' '.join(sql.lower().split())
    
    def _calculate_semantic_similarity(self, sql1: str, sql2: str) -> float:
        """計算SQL語義相似度"""
        # 簡化實現：基於關鍵詞重疊
        words1 = set(self._normalize_sql(sql1).split())
        words2 = set(self._normalize_sql(sql2).split())
        
        if not words1 and not words2:
            return 1.0
        if not words1 or not words2:
            return 0.0
        
        intersection = words1.intersection(words2)
        union = words1.union(words2)
        
        return len(intersection) / len(union)
    
    def _sql_similarity(self, sql1: str, sql2: str) -> float:
        """計算SQL相似度"""
        return self._calculate_semantic_similarity(sql1, sql2)
    
    def _compare_execution_results(self, generated_sql: str, expected_sql: str) -> float:
        """比較執行結果"""
        try:
            conn = sqlite3.connect('sample_ecommerce.db')
            
            # 執行兩個SQL
            result1 = pd.read_sql_query(generated_sql, conn)
            result2 = pd.read_sql_query(expected_sql, conn)
            
            conn.close()
            
            # 比較結果
            if result1.equals(result2):
                return 1.0
            elif len(result1) == len(result2) and len(result1.columns) == len(result2.columns):
                return 0.5  # 部分匹配
            else:
                return 0.0
                
        except Exception:
            return 0.0
    
    def generate_evaluation_report(self, test_cases: List[Dict]) -> str:
        """生成評估報告"""
        retrieval_eval = self.evaluate_retrieval(test_cases)
        generation_eval = self.evaluate_generation(test_cases)
        e2e_eval = self.evaluate_end_to_end(test_cases)
        
        report = "\n" + "="*50 + "\n"
        report += "TextSQL RAG Pipeline 評估報告\n"
        report += "="*50 + "\n"
        
        report += "\n📊 檢索性能:\n"
        report += f"  平均檢索分數: {retrieval_eval['avg_retrieval_score']:.3f}\n"
        report += f"  平均精確度: {retrieval_eval['avg_precision']:.3f}\n"
        
        report += "\n🔧 生成性能:\n"
        report += f"  精確匹配率: {generation_eval['exact_match_rate']:.3f}\n"
        report += f"  語法有效率: {generation_eval['syntax_valid_rate']:.3f}\n"
        report += f"  平均語義相似度: {generation_eval['avg_semantic_similarity']:.3f}\n"
        
        report += "\n🚀 端到端性能:\n"
        report += f"  執行成功率: {e2e_eval['execution_success_rate']:.3f}\n"
        report += f"  結果准確率: {e2e_eval['result_accuracy_rate']:.3f}\n"
        
        # 總體評分
        overall_score = np.mean([
            retrieval_eval['avg_retrieval_score'],
            generation_eval['syntax_valid_rate'],
            e2e_eval['execution_success_rate']
        ])
        
        report += f"\n⭐ 總體評分: {overall_score:.3f}\n"
        
        return report

# 創建評估器
evaluator = RAGEvaluator(pipeline)

# 運行評估
print("🔍 開始評估流水線性能...")
evaluation_report = evaluator.generate_evaluation_report(training_data)
print(evaluation_report)

## 10. 實際應用案例

### 互動式查詢界面

In [ ]:
# 互動式查詢界面
class InteractiveQueryInterface:
    def __init__(self, pipeline: TextSQLRAGPipeline):
        """初始化互動式界面"""
        self.pipeline = pipeline
        self.session_history = []
    
    def start_session(self):
        """開始互動會話"""
        print("\n🎯 歡迎使用 TextSQL RAG 查詢系統！")
        print("輸入自然語言查詢，系統將生成對應的SQL語句並執行")
        print("輸入 'help' 查看幫助，輸入 'quit' 退出系統\n")
        
        while True:
            try:
                user_input = input("🔍 請輸入您的查詢: ").strip()
                
                if user_input.lower() == 'quit':
                    print("👋 感謝使用，再見！")
                    break
                elif user_input.lower() == 'help':
                    self._show_help()
                    continue
                elif user_input.lower() == 'history':
                    self._show_history()
                    continue
                elif user_input.lower() == 'schema':
                    self._show_schema()
                    continue
                elif not user_input:
                    continue
                
                # 處理查詢
                self._process_interactive_query(user_input)
                
            except KeyboardInterrupt:
                print("\n👋 用戶中斷，再見！")
                break
            except Exception as e:
                print(f"❌ 發生錯誤: {e}")
    
    def _process_interactive_query(self, query: str):
        """處理互動查詢"""
        print(f"\n⏳ 處理中...")
        
        # 執行查詢
        results = self.pipeline.process_query(query, execute_sql=True)
        
        # 記錄會話歷史
        self.session_history.append(results)
        
        # 顯示結果
        self._display_results(results)
    
    def _display_results(self, results: Dict):
        """顯示查詢結果"""
        print("\n" + "="*50)
        print(f"📝 查詢: {results['natural_language_query']}")
        print("="*50)
        
        # 顯示生成的SQL
        generation = results['generation_results']
        print(f"\n💻 生成的SQL:")
        print(f"```sql\n{generation['generated_sql']}\n```")
        
        print(f"\n🔧 生成方法: {generation['method']}")
        print(f"✅ SQL有效性: {generation['is_valid']}")
        
        # 顯示執行結果
        if results['execution_results']:
            execution = results['execution_results']
            if execution['success']:
                print(f"\n🎯 執行結果: 成功返回 {execution['row_count']} 行數據")
                
                if execution['data']:
                    # 將結果格式化為表格
                    df = pd.DataFrame(execution['data'])
                    print("\n📊 查詢結果:")
                    print(df.to_string(index=False))
                else:
                    print("\n📊 查詢結果: 無數據返回")
            else:
                print(f"\n❌ 執行失敗: {execution['error']}")
        
        # 顯示檢索信息
        retrieval = results['retrieval_results']
        if retrieval['similar_examples']:
            best_match = retrieval['similar_examples'][0]
            print(f"\n🔍 最佳匹配示例: {best_match['text']}")
            print(f"📈 相似度: {1-best_match['distance']:.3f}")
    
    def _show_help(self):
        """顯示幫助信息"""
        help_text = """
📚 使用幫助:

基本查詢示例:
  • "顯示所有用戶" - 查詢用戶表
  • "找出最貴的產品" - 按價格排序
  • "統計每個類別的產品數量" - 聚合查詢
  • "顯示張三的所有訂單" - 聯合查詢

特殊命令:
  • help - 顯示此幫助
  • history - 顯示查詢歷史
  • schema - 顯示數據庫結構
  • quit - 退出系統

💡 提示: 盡量使用自然語言描述您的查詢需求！
"""
        print(help_text)
    
    def _show_history(self):
        """顯示查詢歷史"""
        if not self.session_history:
            print("\n📝 暫無查詢歷史")
            return
        
        print(f"\n📝 查詢歷史 (共 {len(self.session_history)} 條):")
        print("-" * 60)
        
        for i, result in enumerate(self.session_history[-5:], 1):  # 只顯示最近5條
            query = result['natural_language_query']
            sql = result['generation_results']['generated_sql']
            success = result['pipeline_success']
            status = "✅" if success else "❌"
            
            print(f"{i}. {status} {query}")
            print(f"   SQL: {sql[:50]}{'...' if len(sql) > 50 else ''}")
            print()
    
    def _show_schema(self):
        """顯示數據庫結構"""
        print("\n🗃️ 數據庫結構:")
        print(self.pipeline.schema.get_schema_info())

# 創建互動界面
interface = InteractiveQueryInterface(pipeline)

# 演示一些自動查詢（而不是真正的互動模式）
print("\n🎯 TextSQL RAG 系統演示")
print("以下是一些自動執行的查詢示例:\n")

demo_queries = [
    "查看所有用戶的信息",
    "找出價格最高的三個產品",
    "統計每個用戶的訂單總數"
]

for query in demo_queries:
    print(f"🔍 查詢: {query}")
    interface._process_interactive_query(query)
    print("\n" + "-"*60 + "\n")

print("\n✨ 演示完成！要開始真正的互動模式，請取消註釋下面的代碼：")
print("# interface.start_session()")

## 總結與優化建議

### 學習總結

通過本筆記本，我們完整實現了一個 TextSQL RAG 流水線，包括：

1. **數據預處理**: 創建示例數據庫和訓練數據
2. **向量化**: 使用句子嵌入模型將文本轉換為向量
3. **檢索系統**: 基於語義相似度檢索相關示例
4. **SQL生成**: 結合檢索結果和模板生成SQL
5. **完整流水線**: 整合所有組件
6. **評估系統**: 多維度評估性能
7. **實際應用**: 互動式查詢界面

### 優化建議

1. **模型優化**:
   - 使用更強大的嵌入模型（如 OpenAI embeddings）
   - 實現微調機制以適應特定領域

2. **檢索優化**:
   - 實現混合檢索（語義+關鍵詞）
   - 添加重排序機制

3. **生成優化**:
   - 集成大語言模型（如 GPT-4）
   - 實現更複雜的SQL模板

4. **系統優化**:
   - 添加緩存機制
   - 實現分佈式部署
   - 增強錯誤處理

### 後續學習方向

1. 深入學習 Transformer 架構
2. 探索更高級的 RAG 技術
3. 學習數據庫優化技術
4. 研究多模態 RAG 系統

In [ ]:
# 保存流水線狀態和結果
import pickle

def save_pipeline_state():
    """保存流水線狀態"""
    state = {
        'query_history': pipeline.query_history,
        'pipeline_stats': pipeline.get_pipeline_stats(),
        'training_data': training_data,
        'schema_info': schema.get_schema_info()
    }
    
    with open('pipeline_state.pkl', 'wb') as f:
        pickle.dump(state, f)
    
    # 也保存為JSON格式便於查看
    json_state = {
        'pipeline_stats': pipeline.get_pipeline_stats(),
        'training_data': training_data,
        'schema_info': schema.get_schema_info()
    }
    
    with open('pipeline_summary.json', 'w', encoding='utf-8') as f:
        json.dump(json_state, f, ensure_ascii=False, indent=2)
    
    print("✅ 流水線狀態已保存")

# 保存狀態
save_pipeline_state()

print("\n🎉 TextSQL RAG Pipeline 學習筆記完成！")
print("\n📝 學習成果:")
print(f"  • 處理了 {len(pipeline.query_history)} 個查詢")
print(f"  • 成功率: {pipeline.get_pipeline_stats().get('success_rate', 0):.1%}")
print(f"  • 平均檢索分數: {pipeline.get_pipeline_stats().get('avg_retrieval_score', 0):.3f}")
print("\n🚀 您現在可以:")
print("  1. 在 Kaggle 環境中運行此筆記本")
print("  2. 修改數據庫結構和訓練數據")
print("  3. 優化檢索和生成算法")
print("  4. 集成更強大的語言模型")
print("\n💡 建議下一步: 嘗試在真實數據集上測試此系統！")